In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import animation  
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import manifold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
dao_data = pd.read_csv("tokens.csv")
dao_data = dao_data.copy().dropna()

dao_data.pop("_id/$oid")
dao_data.pop("name")
dao_data.pop("symbol")
dao_data.pop("contractAddress")
dao_data.pop("chain/$oid")
dao_data.pop("logo")
dao_data.pop("decimal")
dao_data.pop("financialData/lastUpdated/$date")
dao_data.pop("live")
dao_data.pop("createdAt/$date")
dao_data.pop("updatedAt/$date")
dao_labels = dao_data.pop("__v")

dao_data.head(n = 5)

,financialData/price,financialData/volume24h,financialData/volumeChange24h,financialData/percentChange1h,financialData/percentChange24h,financialData/percentChange7d,financialData/percentChange30d,financialData/percentChange60d,financialData/percentChange90d,financialData/marketCap,financialData/marketCapDominance,financialData/fullyDilutedMarketCap
1,107.118305,2.394103e+07,98.6279,-2.590717,2.321428,9.309774,-20.503689,-29.946043,-47.405427,1.281386e+08,0.0064,1.674200e+08
2,2.012771,2.222964e+07,-26.2485,-0.083925,-0.819049,2.861661,1.968099,0.515132,-15.602256,1.252114e+08,0.0063,1.287623e+08
3,2.604602,4.620275e+08,-35.9028,0.081143,-1.606518,6.577040,-0.748148,25.014858,-30.904646,4.794600e+09,0.2399,5.713784e+09
4,0.000005,9.751028e+04,102.6711,-3.107993,-1.089080,-20.801173,23.767579,37.165415,37.165415,0.000000e+00,0.0000,4.969091e+06
5,4.283644,7.365072e+05,3.7986,-0.345016,-2.939907,8.500397,-31.746554,-32.329757,-40.635534,8.173324e+06,0.0000,8.191968e+06


In [3]:
x_inputs = {}

for col_name, column in dao_data.items():
    dtype = column.dtype
    if dtype == object:
        dtype = tf.string
    else:
        dtype = tf.float32
    x_inputs[col_name] = tf.keras.Input(shape = (1,), name = col_name, dtype = dtype)

x_inputs

{'financialData/price': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/price')>,
 'financialData/volume24h': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/volume24h')>,
 'financialData/volumeChange24h': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/volumeChange24h')>,
 'financialData/percentChange1h': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/percentChange1h')>,
 'financialData/percentChange24h': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/percentChange24h')>,
 'financialData/percentChange7d': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/percentChange7d')>,
 'financialData/percentChange30d': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/percentChange30d')>,
 'financialData/percentChange60d': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData

In [5]:
num_inputs = {name: input for name, input in x_inputs.items() if input.dtype == tf.float32}

x = tf.keras.layers.Concatenate()(list(num_inputs.values()))
normalization = tf.keras.layers.experimental.preprocessing.Normalization()
normalization.adapt(np.array(dao_data[num_inputs.keys()]))
all_num_inputs = normalization(x)

preprocessed_inputs = [all_num_inputs]

preprocessed_inputs

[<KerasTensor: shape=(None, 12) dtype=float32 (created by layer 'normalization')>]

In [7]:
for name, input in x_inputs.items():
    if input.dtype != tf.string:
        continue

    str_lookup = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary = np.unique(dao_data[col_name]))
    one_hot_enc = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens = str_lookup.vocab_size())

    x = str_lookup(input)
    x = one_hot_enc(x)
    preprocessed_inputs.append(x)

preprocessing_layer = tf.keras.Model(x_inputs, preprocessed_inputs)
preprocessed_inputs

[<KerasTensor: shape=(None, 12) dtype=float32 (created by layer 'normalization')>]